In [14]:
# Required packages for the project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Data download
data_RPS = pd.read_csv("RPS_Cont_2013-2020.csv", low_memory=False)


In [ ]:
# Replacing values in the Status column
cleanup_nums = {"Status": {"NC": 0, "PCS": 1, "EFOS": 1}}
data_RPS = data_RPS.replace(cleanup_nums)

In [15]:
# Function to display all columns
pd.set_option('display.max_columns', None)

In [16]:
# Creating a copy of the database for each model
data_normal = data_RPS.copy()
data_redf = data_RPS.copy()


# Base de datos normal

In [17]:
# Removing columns that will not be used. In this case (DepID, ProvContID)
features = ['GO.APF', 'GO.GE', 'GO.GM', 'PC.N', 'PC.I', 'PC.ITLC', 'CT.OP', 'CT.S', 'CT.AD', 'CT.AR', 'CT.SLAOP',
            'PT.AD', 'PT.I3P', 'PT.LP', 'S.NOM', 'S.MED', 'S.PEQ', 'S.MIC', 'S.NA',
            'BeginningWeek', 'EndingWeek', 'EBWeeks', 'Spending', 'T.Cont', 'T.Cont.Max', 'T.Spending',
            'T.Spending.Max', 'T.AD', 'ActiveWeeks', 'CPW', 'SPW', 'Fav', 'RAD', 'Status']
data_normal = data_normal[features]


# RedFlags

•		Percentage of split contracts per year

In [19]:
# Filter the main database for non-competitive contracts, i.e., Direct Award and Restricted Invitation
data_RPS_without_PTLP = data_redf[(data_redf["PT.AD"] == 1) | (data_redf["PT.I3P"] == 1)]

# Create a pivot table to identify the occurrences of contracts in the same starting week and year
data_RPS_without_PTLP_sameWandY = data_RPS_without_PTLP.groupby(['ProvContID', 'DepID', 'Year', 'BeginningWeek'])['BeginningWeek'].count().reset_index(name="Same_BegWeekandYear_Count")

# Filter by suppliers and contractors who have had more than one contracting process in the same week of the same year
data_RPS_without_PTLP_sameWandY = data_RPS_without_PTLP_sameWandY[data_RPS_without_PTLP_sameWandY["Same_BegWeekandYear_Count"] > 1]

# Select the columns of interest
data_RPS_without_PTLP_sameWandY = data_RPS_without_PTLP_sameWandY[["ProvContID", "Year", "DepID", "BeginningWeek", "Same_BegWeekandYear_Count"]]

# This table shows the number of contracting processes by suppliers and contractors per year
ProvContID_tot_perY = data_redf.groupby(["ProvContID", "Year"]).count()

# Reset the index
ProvContID_tot_perY.reset_index(inplace=True)

# Remove the unwanted columns
ProvContID_tot_perY = ProvContID_tot_perY[['ProvContID', 'Year', 'Status']]

# Rename the 'Status' column to 'Tot_ProvContID_per_Year' (Total Bidding Processes per Supplier and Year)
ProvContID_tot_perY = ProvContID_tot_perY.rename(columns={'Status': 'Tot_ProvContID_per_Year'})

# Merge with the main database
data_redf = pd.merge(data_redf, ProvContID_tot_perY, on=["ProvContID", "Year"], how="left")

# Merge the databases
data_redf = pd.merge(data_redf, data_RPS_without_PTLP_sameWandY, on=["ProvContID", "DepID", "Year", "BeginningWeek"], how="left")

# Calculate the percentage of contracts that occurred on the same day and year relative to the total contracts of a supplier per year
data_redf["PSC"] = data_redf["Same_BegWeekandYear_Count"] / data_redf["Tot_ProvContID_per_Year"]

# Remove the columns used for calculation
data_redf.drop(["Tot_ProvContID_per_Year", "Same_BegWeekandYear_Count"], axis=1, inplace=True)

# Fill the null values with 0
data_redf["PSC"] = data_redf["PSC"].fillna(0)

data_redf.shape

(1540386, 38)


•	Percentage of non-open procedures by government agency

In [20]:
# Filter out data that is not PT.AD (Direct Award)
data_PT_AD_1 = data_redf[data_redf["PT.AD"] == 1]

# Group by DepID and count the number of PT.AD to determine the total number of Direct Awards per institution and year
data_PT_AD_1 = data_PT_AD_1.groupby(['DepID', 'PT.AD', 'Year'])['PT.AD'].count().reset_index(name='Total_AD_DepID')

# Remove the PT.AD column
data_PT_AD_1.drop(['PT.AD'], axis=1, inplace=True)

# This table tells us the number of bids per department and year.
# Group by DepID and Year.
DepID_tot = data_redf.groupby(['DepID', 'Year']).count()

# Reset the index
DepID_tot.reset_index(inplace=True)

# Remove the unwanted columns
DepID_tot = DepID_tot[['DepID', 'Year', 'Status']]

# Rename the Status column to Tot_DepID_per_Year
DepID_tot = DepID_tot.rename(columns={'Status': 'Tot_DepID_per_Year'})

# Merge the two tables
DepID_AD = pd.merge(data_PT_AD_1, DepID_tot, on=['DepID', 'Year'], how='left')

# Create a new column that represents the percentage of Direct Awards per department and year
DepID_AD = DepID_AD.assign(FSBG=lambda x: x.Total_AD_DepID / x.Tot_DepID_per_Year)

# Remove the unwanted columns
DepID_AD = DepID_AD.drop(['Total_AD_DepID', 'Tot_DepID_per_Year'], axis=1)

# Merge the previous table with the original data table
data_redf = pd.merge(data_redf, DepID_AD, on=['DepID', 'Year'], how='left')

# Replace NaN values with 0
data_redf["FSBG"] = data_redf["FSBG"].fillna(0)

data_redf.shape



(1540386, 39)

•	Percentage of non-open procedures by supplier

In [21]:
# Using the same PT.AD filter, we perform the same process but with ProvContID
data_PT_AD_1 = data_redf[data_redf["PT.AD"] == 1]

# Group by ProvContID and count the number of PT.AD to determine the total number of Direct Awards per provider and year
ProcContID_PT_AD_Tot = data_PT_AD_1.groupby(['ProvContID', 'PT.AD', 'Year'])['PT.AD'].count().reset_index(name='Total_AD_ProvContID')

# Remove the PT.AD column
ProcContID_PT_AD_Tot.drop(['PT.AD'], axis=1, inplace=True)

# This table tells us the number of bids per provider and year.
# Group by ProvContID and Year.
ProvContID_tot = data_redf.groupby(['ProvContID', 'Year']).count()

# Reset the index
ProvContID_tot.reset_index(inplace=True)

# Remove the unwanted columns
ProvContID_tot = ProvContID_tot[['ProvContID', 'Year', 'Status']]

# Rename the Status column to Num_ProvContID_per_Year
ProvContID_tot = ProvContID_tot.rename(columns={'Status': 'Num_ProvContID_per_Year'})

# Merge the two tables
data = pd.merge(ProcContID_PT_AD_Tot, ProvContID_tot, on=['ProvContID', 'Year'], how='left')

# Create a new column that represents the percentage of Direct Awards per provider and year
data["F_NonOpen_ProvCont"] = data["Total_AD_ProvContID"] / data["Num_ProvContID_per_Year"]

# Drop the unwanted columns
data = data.drop(['Total_AD_ProvContID', 'Num_ProvContID_per_Year'], axis=1)

# Merge the previous table with the original data table
data_redf = pd.merge(data_redf, data, on=['ProvContID', 'Year'], how='left')

# Remove the unwanted columns
# data_redf = data_redf.drop(['Total_AD_ProvContID', "Num_AD_ProvContID_per_Year_y"], axis=1)

# Replace NaN values with 0
data_redf["F_NonOpen_ProvCont"] = data_redf["F_NonOpen_ProvCont"].fillna(0)

data_redf.shape



(1540386, 40)

•	Frequency of contracts won: 

In [22]:
# Create a pivot table to determine the number of contracts per department and year
ProvContID_perYear = data_redf.groupby(['DepID', 'Year', 'ProvContID'])['ProvContID'].count().reset_index(name='Total_ProvContID')

# With this pivot table, we can find the maximum number of contracts per department and year
Max_ProvContID_perYear = ProvContID_perYear.groupby(['DepID', 'Year'])['Total_ProvContID'].agg(max).reset_index(name='Max_Total_ProvContID')

# Merge the two tables
data2 = pd.merge(ProvContID_perYear, Max_ProvContID_perYear, on=['DepID', 'Year'], how='left')

# Create a new column that represents the percentage of contracts won by department and year
data2["F_of_Contr_Won_IMCO"] = (data2["Total_ProvContID"] * 100) / data2["Max_Total_ProvContID"]

# Remove the unwanted columns
data2.drop(['Total_ProvContID', 'Max_Total_ProvContID'], axis=1, inplace=True)

# Merge the previous table with the original data table
data_redf = pd.merge(data_redf, data2, on=['DepID', 'Year', 'ProvContID'], how='left')

# Uncomment the line below if you want to fill the NaN values with 0
# data_redf["Freq_of_Contr_Won_IMCO"] = data_redf["Freq_of_Contr_Won_IMCO"].fillna(0)

data_redf.shape



(1540386, 41)

•	Amount of Contracted by the supplier

In [23]:
# Create a pivot table to determine the total spending per department, company, and year
Sum_Spend = data_redf.groupby(['DepID', 'Year', 'ProvContID'])['Spending'].sum().reset_index(name='Spending')

# Create a pivot table to determine the maximum spending per department and year
Sum_Spend_max = Sum_Spend.groupby(['DepID', 'Year'])['Spending'].max().reset_index(name='Max_Spending')

# Merge the two tables
data3 = pd.merge(Sum_Spend, Sum_Spend_max, on=['DepID', 'Year'], how='left')

# Create a new column that represents the standardized total amount of contracts for company i in year t, awarded by company d
data3["Am_Cont_by_Sup"] = (data3["Spending"] * 100) / data3["Max_Spending"]

# Remove the unwanted columns
data3.drop(['Spending', 'Max_Spending'], axis=1, inplace=True)

# Merge the previous table with the original data table
data_redf = pd.merge(data_redf, data3, on=['DepID', 'Year', 'ProvContID'], how='left')

data_redf.shape

(1540386, 42)

In [24]:
# Define the columns to keep
features = ['Status', 'GO.APF', 'GO.GE', 'GO.GM', 'PC.N', 'PC.I', 'PC.ITLC', 'CT.OP', 'CT.S', 'CT.ADQ', 'CT.AR', 'CT.SLAOP', 'PT.AD', 'PT.I3P', 'PT.LP', 'S.NOM', 'S.MED', 'S.PEQ', 'S.MIC', 'S.NA', 'Year', 'BeginningWeek', 'EndingWeek', 'EBWeeks', 'Spending', 'T.Cont', 'T.Cont.Max', 'T.Spending', 'T.Spending.Max', 'T.AD', 'ActiveWeeks', 'CPW', 'SPW', 'Fav', 'RAD', 'PSC', 'FSBG', 'F_NonOpen_ProvCont', 'F_of_Contr_Won_IMCO', 'Am_Cont_by_Sup']

# Keep only the selected columns in the data_redf DataFrame
data_redf = data_redf[features]



In [28]:
data_normal.to_csv("RPS_N.csv")
data_redf.to_csv("RPS_RedFlags.csv")

In [29]:
# Save data_normal as RPS_N.csv
data_normal.to_csv("RPS_N.csv", index=False)

# Save data_redf as RPS_RedFlags.csv
data_redf.to_csv("RPS_RedFlags.csv", index=False)


(1540386, 34)
(1540386, 40)
